# TFRecord
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- Tensorflow에서 제공하는 데이터셋을 저장방식.
    - 데이터 양이 많을 경우 이를 Binary로 Serialization(직렬화)하여 하나의 파일로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer 형태로 Serialization을 수행해서 저장할 수 있는 TFRecords 파일 포맷 형태를 지원한다. 
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecords 파일에 저장할 수 있다.
- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.
- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, byte 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환
- tf.tran.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```

In [1]:
import tensorflow as tf

In [10]:
# tf.train.Feature 객체를 생성하는 (기본타입의 값들을 Feature로 변환하는) 함수 구현
def _bytes_feature(value):
    """
    value로 string, bytes를 받아서 BytesList로 변환하는 함수
    """
    # value가 Tensor(텐서플로우의 배열 타입) 타입인 경우 ndarray로 변환 => binary파일
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """
    float타입의 value를 받아서 FloatList 변환 함수
    """
    return tf.train.Feature(float_list = tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """
    int, uint, bool타입의 value를 받아서 Int64List로 변환하는 함수
    """
    return tf.train.Feature(int64_list = tf.train.Int64List(value=[value]))

In [2]:
type(tf.constant(0))

tensorflow.python.framework.ops.EagerTensor

In [13]:
v = _int64_feature(20)
print(type(v), v)

<class 'tensorflow.core.example.feature_pb2.Feature'> int64_list {
  value: 20
}



In [11]:
v = _float_feature(56.7)
print(type(v), v)

<class 'tensorflow.core.example.feature_pb2.Feature'> float_list {
  value: 56.7
}



In [12]:
v = _int64_feature(False) # bool: true-1, false-0
print(type(v), v)

<class 'tensorflow.core.example.feature_pb2.Feature'> int64_list {
  value: 0
}



In [14]:
v = _bytes_feature(b"hello") # 문자열, byte=> bytes타입을 전달, 문자열앞에 b추가
print(type(v), v)

<class 'tensorflow.core.example.feature_pb2.Feature'> bytes_list {
  value: "hello"
}



In [15]:
s = "홍길동" # 일반 string일 경우 => byte타입으로 변환뒤에 BytesList로 변환
_bytes_feature(s.encode('utf-8'))

bytes_list {
  value: "\355\231\215\352\270\270\353\217\231"
}

In [16]:
s = "Hong" # 일반 string일 경우 => byte타입으로 변환뒤에 BytesList로 변환
_bytes_feature(s.encode('utf-8'))

bytes_list {
  value: "Hong"
}

In [17]:
# binary파일 (이미지)
with open('tiger.jpg', 'rb') as f:
    img = f.read()
type(img)

bytes

In [18]:
v = _bytes_feature(img)
print(type(v), v)

<class 'tensorflow.core.example.feature_pb2.Feature'> bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000\204\000\n\007\010\026\025\025\030\026\026\026\031\030\030\032\031\031\030\030\034\034\034\030\032\032\032\030\031\034\034\032\030\030\034\032\034!.%\034\036+!\030\030&8&+/1555\032$;@;4?.451\001\014\014\014\020\017\020\036\022\022\0374+$$44444444464444444444444444444444444444444444444444\377\300\000\021\010\000\271\001\021\003\001\"\000\002\021\001\003\021\001\377\304\000\033\000\000\002\002\003\001\000\000\000\000\000\000\000\000\000\000\000\004\005\003\006\000\002\007\001\377\304\000=\020\000\002\001\002\004\004\003\006\004\004\005\004\003\000\000\000\001\002\021\000\003\004\022!1\005AQa\006\"q\0232\201\221\241\261B\301\321\360\024Rbr\007#\262\341\361\0253\202\302$%s\377\304\000\031\001\000\003\001\001\001\000\000\000\000\000\000\000\000\000\000\000\001\002\003\000\004\005\377\304\000$\021\000\002\002\002\003\000\002\003\001\001\

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력하기 전에 변환해야 한다.

In [19]:
feature = _float_feature(30.2)
print(feature)

float_list {
  value: 30.2
}



In [21]:
v = feature.SerializeToString() # 출력형태로 변환. String=> bytes
print(type(v), v)

<class 'bytes'> b'\x12\x06\n\x04\x9a\x99\xf1A'


# TFRecord 저장 예제

### tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.


### TFRecord로 저장할 toy dataset을 생성

In [23]:
import numpy as np
N_DATA = 1000 # dataset의 데이터 개수

# bool 1000생성
feature0 = np.random.choice([False, True], N_DATA)
# 정수 1000
feature1 = np.random.randint(0,5,N_DATA)
# string 1000
str_list = np.array([b'lion',b'tiger',b'cat',b'dog',b'bear'])
feature2 = str_list[feature1]
#float 1000
feature3 = np.random.randn(N_DATA)

feature0.shape,feature1.shape,feature2.shape,feature3.shape

((1000,), (1000,), (1000,), (1000,))

In [25]:
def serialize_example(feature0,feature1,feature2,feature3):
    """
    하나의 데이터의 속성(Feature) 값들을 받아서 Example을 생성한뒤 그 Example을 출력가능한 bytes로 만들어 반환(SerializetoString()이용)
    feature0: bool,
    feature1: int,
    feature2: bytes(string),
    feature3: float
    """
    # feature들을 dictionary로 생성
    feature = {
        "feature0":_int64_feature(feature0),
        "feature1":_int64_feature(feature1),
        "feature2":_bytes_feature(feature2),
        "feature3":_float_feature(feature3),
    }
    #feature들을 가진 tf.train.Example 객체를 생성
    example = tf.train.Example(features=tf.train.Features(feature=feature))
#     print(type(example))
    
    # Example을 TFRecord에 저장하기 위한 형태인 bytes로 변환 => SerializeToString()이용
    return example.SerializeToString()

### 출력 처리
- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [26]:
import os
# TFRecord 파일 저장 디렉토리 생성
if not os.path.isdir('tfrecord'):
    os.mkdir('tfrecord')

In [27]:
# TFRecordWriter 객체를 생성 -> TFRecord파일로 직렬화된 Example을 출력하는 메소드 제공.
tfrecord_file_path = './tfrecord/data.tfr' # 확장자: tfr, record, tfrecord
tfrecord_writer = tf.io.TFRecordWriter(tfrecord_file_path)

In [29]:
for data in zip(feature0,feature1,feature2,feature3):
    sv = serialize_example(bool(data[0]),data[1],data[2],data[3])
    tfrecord_writer.write(sv)

tfrecord_writer.close() #출력 Stream 연결 닫기

<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.exam

<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.exam

<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.example.example_pb2.Example'>
<class 'tensorflow.core.exam

# TFRecord파일 읽기 및 역직렬화(Deserialize)
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [31]:
def _parse_function(tfrecord_seralized):
    """
    serialize(직렬화-bytes)된 Example데이터 하나를 받아서 역직렬화 한뒤에 반환하는 함수
    [매개변수]
        tfrecord_serialized: 직렬화된 Example
    [반환값]
        Example구성 Feature들.(feature0,1,2,3)
    """
    # 역직렬화 해서 읽어온 Feature들을 저장할 Feature들을 dictionary로 구성
    # 이름: 저장할때 사용한 name, value: 읽어온 Feature를 저장할 빈 Feature(역직렬화할 값의 타입을 선언한다.)
    feature = {
        'feature0':tf.io.FixedLenFeature([], tf.int64),
        'feature1':tf.io.FixedLenFeature([], tf.int64),
        'feature2':tf.io.FixedLenFeature([], tf.string),
        'feature3':tf.io.FixedLenFeature([], tf.float32),
        }
    parsed_features = tf.io.parse_single_example(tfrecord_seralized, feature)
    # tf.cast(값, dtype) 값의 데이터 타입을 지정한 dtype으로 변환
    feature0 = tf.cast(parsed_features['feature0'], tf.bool)
    feature1 = tf.cast(parsed_features['feature1'], tf.int64)
    feature2 = tf.cast(parsed_features['feature2'], tf.string)
    feature3 = tf.cast(parsed_features['feature3'], tf.float32)
    
    return feature0,feature1,feature2,feature3

In [32]:
# TFRecord파일 읽기
dataset = tf.data.TFRecordDataset(tfrecord_file_path).map(_parse_function)


In [34]:
for data in dataset.take(3):
#     print(data)
    print(data[0].numpy(),data[1].numpy(),data[2].numpy(),data[3].numpy())

True 4 b'bear' -0.90646476
False 3 b'dog' -0.7493212
False 2 b'cat' -0.07792034
